# IBM APPLIED DATA SCIENCE CAPSTONE - THE BATTLE OF NEIGHBORHOODS

## TABLE OF CONTENT
+ **THE BUSINESS CASE**
+ **THE DATA**
+ **THE LIBRARIES**

## THE BUSINESS CASE 
My fiance and I love pizza. We eat it minimum once a week and we are pretty picky about it. The best Pizza place in our neighbourhood is called [L'Osteria](https://losteria.net/en/). Pizzas are huge, drinks are decent, atmosphere is great and prices are reasonable - perfect place for young people like us!
<br>
<br>
Your probably now wonder what this has to do with the Capstone Project for the [IBM - Data Science Professional Certficiate](https://www.coursera.org/professional-certificates/ibm-data-science). L'Osteria is a restaurant chain for italian food, founded in 1999 in Germany. The chain currently has 139 restaurants in Europe, of which the majority is located in Germany. As the chain is looking to expand further, it continuously needs to scout new locations for their next restaurants. As that can be an extremely difficult and long process, this Capstone Project aims to build an analysis tool that recommends neighbourhoods which could be a good fit for new L'Osteria locations. In result, the process of scouting new neihbourhoods should be more time and cost efficient.
<br>
<br>
The tool is going to be based on a k-means clustering algorithm that uses data from Foursquare, Facebook and governmental websites. As my ressources for this project are limited, the tool will be focused on Germany only. However, the approach could be adapted to any other country or region, if enough ressources are provided. Further ressources would also enable adding more socio-economic data which would make the algorithm more precise. 

## THE DATA 
As already mentioned above, we are going to use data from different sources. In this section, I will provide you with a detailed description for each data source used and provide you the code you need to download/upload the data.
<br>
<br>
### FOURSQUARE DATA
[Foursquare](https://foursquare.com/) is an app that lets its users rank all different kinds of venues all over the world. By doing so,Foursquare has build one of the most comprehensive geospatial data sets in the world. The data set covers an extremely large amount of venue and user data that allows you to access everything from a venue's name, location or menu up to user ratings, comments or pictures of each venue. And the best part: Foursquare provides developers an [API](https://developer.foursquare.com/) that is (in limits) free to use! 
<br>
<br>
Even though Foursquare provides an incredible variety of data, we will use the venue categories (e.g. sushi restaurant, park, sports club, italian restaurant, etc.) for our analysis only. While it would most likely increase the success of our algorithm to include more information such as rankings or types of user data, we will not include this data due to ressource limitations.
<br>
<br>
To being able to access the Foursquare data during our analysis, you must [sign up](https://developer.foursquare.com/) for a developer account with Foursquare first. Afterwards you need to log into your account and navigate to "My Apps". Here, you can create yourself your own client ID and secret. Save these credentials as txt files and name them according to the code below.
<br>
Once you have created your credentials, you need to create an access token. For that, just follow [this guide](https://developer.foursquare.com/docs/places-api/authentication/) and save the access token in a txt file with the name provided below.

In [18]:
# Import working library
import pandas as pd

# Set credentials
client_id = list(pd.read_csv('FOURSQUARE_CLIENT_ID.txt').columns)[0]
client_secret = list(pd.read_csv('FOURSQUARE_CLIENT_SECRET.txt').columns)[0]
access_token = list(pd.read_csv('FOURSQUARE_ACCESS_TOKEN.txt').columns)[0]

# Check out
print("Foursquare credentials loaded.")

Foursquare credentials loaded.


You are now set to use the Foursquare API! We will use these credentials at a later point. For now, we continue with our next data source.

### FACEBOOK POPULATION DATA
Of all the bad Corona did to us, it also had a good side effect: The big data miners of the world opened up and shared some of their digital gold with the rest of us. So did Facebook by publishing large datasets based on their app usage. You can find a detailed description of their "Data for Good" project under this [link](https://dataforgood.fb.com/). Most of the project's data is publicly accessible via [The Humanitarian Data Exchange](https://data.humdata.org/organization/facebook) or [AWS](https://registry.opendata.aws/dataforgood-fb-hrsl/).
<br>
<br>
One of the datasets provided by Facebook contains population density information. According to Facebook, it is the world's most accurate population dataset. It is divided by countries, each containing seven datasets for the distribution of various populations in relation to their respective coordinates: 
<br>
<br>
1. Overall population density 
<br>
2. Women
<br>
3. Men
<br> 
4. Children (ages 0-5) 
<br>
5. Youth (ages 15-24) 
<br>
6. Elderly (ages 60+) 
<br>
7. Women of reproductive age (ages 15-49)
<br>
<br>
Due to limited computing ressources, we will focus on the German datasets for women and men only. You can download them under the following links. Alternatively, you can use the files in my Github repository.
<br>
<br>
1. [DEU_men.csv.zip](https://data.humdata.org/dataset/7d08e2b0-b43b-43fd-a6a6-a308f222cdb2/resource/9b14d65e-ebe1-4509-8680-1f21becc75d9/download/deu_men_2019-08-03_csv.zip)
<br>
2. [DEU_women.csv.zip](https://data.humdata.org/dataset/7d08e2b0-b43b-43fd-a6a6-a308f222cdb2/resource/26b8ec5b-e40b-40af-a4fc-fcd85bfb0818/download/deu_women_2019-08-03_csv.zip)
<br>
<br>
Once downloaded, we can execute the code below. It will load the data, transform and standardize it. Furthermore, we are merging the two datasets to receive one comprehensive population dataset. Finally, we will add a total population column to our dataset which is the sum of all male and female population. 

In [22]:
# Load density data
male_population = pd.read_csv('DEU_men_2019-08-03.csv')
female_population = pd.read_csv('DEU_women_2019-08-03.csv')

# Standardize columns
male_population.rename(columns={"latitude":"LATITUDE", "longitude":"LONGITUDE", "population":"MALE_POPULATION"}, inplace=True)
female_population.rename(columns={"latitude":"LATITUDE", "longitude":"LONGITUDE", "population":"FEMALE_POPULATION"}, inplace=True)

# Merge on geolocation
population = male_population.merge(female_population, how='outer', on=['LATITUDE', 'LONGITUDE'])

# Calculate total population column
population['TOTAL_POPULATION'] = population['MALE_POPULATION'] + population['FEMALE_POPULATION']

# Visualize
print(population.shape)
population.head(10)

(32433580, 5)


,LATITUDE,LONGITUDE,MALE_POPULATION,FEMALE_POPULATION,TOTAL_POPULATION
0,48.814306,8.459306,1.451906,1.552587,3.004493
1,48.815694,8.480972,1.451906,1.552587,3.004493
2,48.804861,8.445139,1.451906,1.552587,3.004493
3,48.787361,8.449306,1.451906,1.552587,3.004493
4,48.819028,8.464306,1.451906,1.552587,3.004493
5,48.824583,8.415139,1.451906,1.552587,3.004493
6,48.775417,8.455972,1.451906,1.552587,3.004493
7,48.817639,8.482917,1.451906,1.552587,3.004493
8,48.769306,8.432917,1.451906,1.552587,3.004493
9,48.817361,8.485417,1.451906,1.552587,3.004493


### GERMAN BORDER DATA
As our Facebook data compromises over 32 million unique geospatial data points that are randomized, there is no way for us to filter this data according to cities or neighborhoods in Germany. One solution to this problem would be to reverse engineer the coordinates with libraries like geopy. However, the accessible APIs via geopy only allow a limited number of calls per day. For +32 million datapoints it would require month do download all necessary data this way. Of course there is always the option to upgrade to a paid enterprise plan to speed up this process. However, this would require financial ressources that I currently do not have. 
<br>
<br>
Luckily, there is another option! The German government publishes shape files of all municipalities Germany. You might ask yourself now how that is going to help us in regards to neighborhoods. The shapefiles provide us with unique polygons for each municipality that we can use to filter our Facebook data. Once filtered, the amount of coordinates that need to be reverse engineered is going to be much less time and cost intensive.
<br>
<br>
The shape files can be downloaded on this [website](https://gdz.bkg.bund.de/index.php/default/digitale-geodaten/verwaltungsgebiete.html) for free. For this excercise, I have used the [vg250_12-31.utm32s.shape.ebenen](https://daten.gdz.bkg.bund.de/produkte/vg/vg250_ebenen_1231/aktuell/vg250_12-31.utm32s.shape.ebenen.zip) dataset. Once you click the link it will direct you to download the respective zip file. Alternatively, you can access the needed files on the Github repository. 
<br>
<br>
The following code will allow you to load the shape file and convert it into a pandas data frame:

In [5]:
# Import working libraries
import pandas as pd

# Define function to read shape files
def read_shapefile(shp_path):
	"""
	Read a shapefile into a Pandas dataframe with a 'coords' column holding
	the geometry information. This uses the pyshp package.
	"""
    # Import working libraries
	import shapefile

	# Read file, parse out the records and shapes
	sf = shapefile.Reader(shp_path)
	fields = [x[0] for x in sf.fields][1:]
	records = sf.records()
	shps = [s.points for s in sf.shapes()]

	# Write into a dataframe
	df = pd.DataFrame(columns=fields, data=records)
	df = df.assign(coords=shps)

    # Check out
	return df

# Set path
path = r"C:\Users\maurice.buettgenbach\OneDrive - Aquila Capital Management GmbH\Desktop\Desktop\Private\IBM\10_Capstone project\IBM_The_battle_of_neighborhoods\VG250_GEM.shp"

# Run function
municipalities = read_shapefile(path)

#Visualize
print(municipalities.shape)
neighborhood_polygons.head()

(11139, 27)


,ADE,GF,BSG,ARS,AGS,SDV_ARS,GEN,BEZ,IBZ,BEM,...,FK_S3,NUTS,ARS_0,AGS_0,WSK,DEBKG_ID,RS,SDV_RS,RS_0,coords
0,6,4,1,010010000000,01001000,010010000000,Flensburg,Stadt,60,kreisfrei,...,R,DEF01,010010000000,01001000,2008-01-01,DEBKGDL20000E5MA,010010000000,010010000000,010010000000,"[(526513.7529476011, 6075133.41194521), (52654..."
1,6,4,1,010020000000,01002000,010020000000,Kiel,Stadt,60,kreisfrei,...,R,DEF02,010020000000,01002000,2006-01-01,DEBKGDL20000004J,010020000000,010020000000,010020000000,"[(575841.569459631, 6032148.031753651), (57586..."
2,6,4,1,010030000000,01003000,010030000000,Lübeck,Stadt,60,kreisfrei,...,R,DEF03,010030000000,01003000,2006-02-01,DEBKGDL20000DYMA,010030000000,010030000000,010030000000,"[(623056.1506336611, 5983746.445214357), (6231..."
3,6,4,1,010040000000,01004000,010040000000,Neumünster,Stadt,60,kreisfrei,...,R,DEF04,010040000000,01004000,1970-04-26,DEBKGDL20000E4SA,010040000000,010040000000,010040000000,"[(565015.6516448742, 6000637.513467715), (5651..."
4,6,4,1,010510011011,01051011,010510011011,Brunsbüttel,Stadt,61,--,...,R,DEF05,010510011011,01051011,2009-01-01,DEBKGDL20000E2IK,010510011011,010510011011,010510011011,"[(510789.9284805175, 5977425.101718364), (5109..."


### OPEN STREET MAP
To reverse geocode our filtered coordinates, we will use the publicly available geospatial data by [OpenStreetMap](https://wiki.openstreetmap.org/wiki/Main_Page). OpenStreetMap is a non profit organization that collects geospatial data and makes in available to the public. We will use this data to reverse geocode our filtered coordinates. We can do so by using the service [Nominatim](https://wiki.openstreetmap.org/wiki/Nominatim) via the [GeoPy](https://geopy.readthedocs.io/en/stable/) library. More details to this in the library section. 

### L'OSTERIA DATA
Finally, we will use the geospatial data from the L'Osteria website to localize their existing restaurants. As the chain is rather big and it would take too long to extract the coordinates from the html by hand, we will use the [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) library to scrape the address data from the website. We then can use Nominatim to receive their coordinates. The following code scrapes and cleans the data so that we can use it later-on:

In [102]:
# Import working libraries
import requests # Request handling
from bs4 import BeautifulSoup # HTML search
import re # String handling

# Define link
link = "https://losteria.net/de/restaurants/view/list/?tx_losteriarestaurants_restaurantsreopening%5Bfilter%5D%5Bcountry%5D=de&tx_losteriarestaurants_restaurantsreopening%5Bfilter%5D%5BsearchTerm%5D=&&&tx_losteriarestaurants_restaurantsreopening%5Bfilter%5D%5Btype%5D=reservations&"

# Download page
page = requests.get(link)

# Create soup object
soup = BeautifulSoup(page.content, 'html.parser')

# Define results
results = soup.find(id='losteria-restaurants-list-wrapper')

# Find elements
page_elements = results.find_all('div', class_='address')

# Create variable to store data in
restaurants = pd.DataFrame()

# Iterate through found elements
for page_element in tqdm(page_elements):
    # Isolate address
    element = str(page_element)
    # Get rid of unnecessary characters
    element = element.replace("\n", "")
    element = element.replace("\r", "")
    # Get rid of spaces
    element = element.replace(" ", "")
    # Search for address in between <div>
    search_result = re.search(">(.*)<", element)
    # Store search result in new variable
    address = search_result.group(1)
    # Get street
    street = address.split(",")[0]
    # Add space between street name and house number
    street = re.sub(r"([0-9]+(\.[0-9]+)?)",r" \1 ", street).strip()
    # Get ZIP code and city
    address = address.split(",")[1]
    # Isolate ZIP code
    zip_code = re.split('(\d+)', address)[1]
    # Isolate city
    city = re.split('(\d+)', address)[2]
    # Create temporary df
    temp_df = pd.DataFrame()
    # Add data to df
    temp_df['STREET'] = [street]
    temp_df['ZIP_CODE'] = zip_code
    temp_df['CITY'] = city
    # Append to variable
    restaurants = restaurants.append(temp_df)
# Reset index
restaurants.reset_index(drop=True, inplace=True)

# Add spacer for output
print()
print()

# Visualize variable
print("df shape:", restaurants.shape)
restaurants.head(12)

100%|██████████| 12/12 [00:00<00:00, 399.38it/s]

df shape: (12, 3)



,STREET,ZIP_CODE,CITY
0,Gut-Dämme-Straße 1,52070,Aachen
1,Franziskanergasse 5,92224,Amberg
2,Pearl-S.-Buck-Straße 12,86156,Augsburg
3,Albert-Schenavsky-Str. 2,86165,Augsburg
4,Maximilianstrasse 83,95444,Bayreuth
5,Hilda-Geiringer-Weg 4,10557,Berlin
6,AltMahlsdorf 88,12623,Berlin
7,Memhardstraße 3,10178,Berlin
8,Mildred-Harnack-Str. 11,10243,Berlin
9,Savignyplatz 5,10623,Berlin


In [110]:
from contextlib import closing
from selenium.webdriver import Firefox # pip install selenium
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
# use firefox to get page with javascript generated content
with closing(Firefox()) as driver:
    driver.get("https://losteria.net/de/restaurants/view/list/?tx_losteriarestaurants_restaurantsreopening%5Bfilter%5D%5Bcountry%5D=de&tx_losteriarestaurants_restaurantsreopening%5Bfilter%5D%5BsearchTerm%5D=&tx_losteriarestaurants_restaurantsreopening%5Bfilter%5D%5Btype%5D=open&tx_losteriarestaurants_restaurantsreopening%5Bfilter%5D%5BmapView%5D=&cHash=6d126f1cfc4964f9c642d679c3674314")
    button = driver.find_element_by_id('losteria-restaurants-loader')
    button.click()
    # wait for the page to load
    element = WebDriverWait(driver, 10).until(
    EC.invisibility_of_element_located((By.ID, "losteria-restaurants-loader"))
    )
    # store it to string variable
    page_source = driver.page_source

soup = BeautifulSoup(page_source)
items = soup.findAll('div', {"class": "list-item"})
print("items count:",len(items))

WebDriverException: Message: 'geckodriver' executable needs to be in PATH. 


## THE LIBRARIES <a name="THE_LIBRARIES" ></a>

In [1]:
# Data wrangling
import pandas as pd
import re

# URL handling
from urllib.request import urlopen
import json

# Web scraping
import requests
from bs4 import BeautifulSoup

# Geo data
from turfpy.measurement import boolean_point_in_polygon
from geojson import Point, Polygon, Feature

# Visualization
from folium import plugins
from folium.plugins import HeatMap
import plotly.express as px
from tqdm import tqdm

# Check out
print('Libraries imported.')

Libraries imported.


In [ ]:
# point = Feature(geometry=Point((-46.6318, -23.5523)))
# polygon = Polygon(
#     [
#         [
#             (-46.653, -23.543),
#             (-46.634, -23.5346),
#             (-46.613, -23.543),
#             (-46.614, -23.559),
#             (-46.631, -23.567),
#             (-46.653, -23.560),
#             (-46.653, -23.543),
#         ]
#     ]
# )
# boolean_point_in_polygon(point, polygon)

In [138]:
# Create temporary variable
df = population.head(1000)

# Create empty list 
addresses = []

# Create counter
counter = 0

# Set geolocator
geolocator = Nominatim(user_agent="german_italian_restaurant")

# Loop through df and get addresses for coordinates
for i in tqdm(range(len(df))):
    # Extract coordinates
    lat = str(df.iloc[i, 0:1].to_numpy()[0])
    lng = str(df.iloc[i, 1:2].to_numpy()[0])
    # Create string from coordinates
    coordinates = str(lat + ", " + lng)
    # Get location with geopy
    location = geolocator.reverse(coordinates)
    # Store in variable
    addresses.append(location.address)

100%|██████████| 1000/1000 [08:19<00:00,  2.00it/s]


In [135]:
# Create address data
address_data = pd.DataFrame()

# Loop through addresses and append them to adress df
for i in tqdm(range(len(addresses))):
    # Split strings
    temp = addresses[i].split(",")
    # Convert to df
    df = pd.DataFrame(temp)
    # Transpose
    df = pd.DataFrame.transpose(df)
    # Concatenate
    frames = [df, address_data]
    address_data = pd.concat(frames)

100%|██████████| 25/25 [00:00<00:00, 464.47it/s]


In [136]:
# Loop through address df and check for inconsistency
for column in tqdm(address_data):
    # Check for NaN values
    if address_data[column].isnull().values.any():
        # If NaN detected, check each row
        for i in range(len(address_data)):
            # Check for NaN in cell
            if pd.isna(address_data.iloc[i, column]) == True:
                # If NaN in cell, shift row
                address_data.iloc[i] = address_data.iloc[i].shift(1)
    else:
        continue

 44%|████▍     | 11/25 [00:00<00:00, 89.18it/s]


In [137]:
print(address_data.shape)
address_data.head()

(25, 11)


,0,1,2,3,4,5,6,7,8,9,10
0,NaN,NaN,NaN,Bienenzüchterverein Bad Herrenalb,Herrenalb,Bad Herrenalb,Verwaltungsgemeinschaft Bad Herrenalb,Landkreis Calw,Baden-Württemberg,76332,Deutschland
0,NaN,Trampolin oder Pool,Moosbronner Straße,Althof,Bernbach,Bad Herrenalb,Verwaltungsgemeinschaft Bad Herrenalb,Landkreis Calw,Baden-Württemberg,76332,Deutschland
0,NaN,145,Gaistalstraße,Unteres Gaistal,Herrenalb,Bad Herrenalb,Verwaltungsgemeinschaft Bad Herrenalb,Landkreis Calw,Baden-Württemberg,76332,Deutschland
0,NaN,21,Im Wiesengrund,Unteres Gaistal,Herrenalb,Bad Herrenalb,Verwaltungsgemeinschaft Bad Herrenalb,Landkreis Calw,Baden-Württemberg,76332,Deutschland
0,NaN,NaN,28,Wallfahrtstraße,Neusatz,Bad Herrenalb,Verwaltungsgemeinschaft Bad Herrenalb,Landkreis Calw,Baden-Württemberg,76332,Deutschland


In [ ]:
# Set column names
columns = [
    'STREET_NUMBER',
    'STREET',
    'NEIGHBORHOOD',
    'CITY'
    ]